In [ ]:
# !pip uninstall -y cffi
# !pip install --no-cache-dir --force-reinstall cffi==1.15.1
# !pip install mip
!pip install --no-cache-dir --no-deps mip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 187.1 MB/s eta 0:00:00


## LEF DEF Parsing

- Install the LEFDEFParser from the the wheel file : [LEFDEFParser-0.1-cp310-cp310-linux_x86_64.whl](https://github.com/srini229/EE5333_tutorials/blob/master/parser/LEFDEFParser-0.1-cp310-cp310-linux_x86_64.whl)
- Download example LEF and DEF files: [Nangate.lef](https://github.com/srini229/EE5333_tutorials/blob/master/parser/Nangate.lef) and [example.def](https://github.com/srini229/EE5333_tutorials/blob/master/parser/example.def)

    <img src="https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/part/fig/example_cir.png" width=340 height=195 />



In [29]:
!pip install --break-system-packages https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/LEFDEFParser-0.1-cp311-cp311-linux_x86_64.whl
!rm -f *.{lef,def}
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/{Nangate.lef,example.def}
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/sample.{lef,def}

  Using cached https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/LEFDEFParser-0.1-cp311-cp311-linux_x86_64.whl (617 kB)
--2025-01-31 12:58:50--  https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/Nangate.lef
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1083933 (1.0M) [text/plain]
Saving to: ‘Nangate.lef’

Nangate.lef         100%[===================>]   1.03M  --.-KB/s    in 0.02s   

2025-01-31 12:58:50 (42.0 MB/s) - ‘Nangate.lef’ saved [1083933/1083933]

--2025-01-31 12:58:50--  https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/example.def
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 899 [text/plain]
Saving to: ‘

In [30]:
#vertex class; name is the instance name of the gate; area: area of the module and the;
class Vertex:
  def __init__(self, name, area):
    self._name = name
    self._area = area
  def __repr__(self):
    return self._name + f" ({round(self._area,2)})"

# Reads input LEF/DEF files and returns a hypergraph with vertex list V and hyperedge list E;
# each hyperedge in E is a tuple of vertices of type `Vertex` defined above.
def loadNetlist(leffile = None, deffile = None):
  import LEFDEFParser as LDP
  l = LDP.LEFReader()
  areaLookup = dict()
  if leffile:
    l.readLEF(leffile)
    areaLookup = {m.name():(m.xdim()*m.ydim()*1.e-6) for m in l.macros()}
  vertices = dict()
  edges = dict()
  if deffile:
    d = LDP.DEFReader()
    d.readDEF(deffile)
    vertices = {c.name() : Vertex(c.name(), areaLookup.get(c.macro(), None)) for c in d.components()}
    edges = {n.name():[vertices[p[0]] for p in n.pins() if p[0] != 'PIN'] for n in d.nets()}
  delE = list()
  for e in edges:
    if len(edges[e]) <= 1:
      delE.append(e)
  for e in delE: del edges[e]
  return vertices, edges

## $k$-way hypergraph partitioning using ILP
+ Hypergraph $H(V,E)$
+ $x_{v,i}$ is the indicator variable for $v$ being in partition $V_i$
+ $x_{e,i}$ is the indicator variable for $e\in E$ being contained in $V_i$

+ Objective: $\max\limits_{x_{e,i}} \sum\limits_{e\in E}\sum\limits_{i=1}^k x_{e,i}$
+ Subject to constraints:
<ul>
$\begin{align}
x_{v,i} &\in \{0, 1\}, &\forall v \in V, \forall i \in \{1,2,\ldots, k\}\\
x_{e,i} &\in \{0, 1\}, &\forall e \in E, \forall i \in \{1,2,\ldots, k\}\\
\sum\limits_{i=1}^k x_{v,i} &=1 , &\forall v \in V\\
\sum_{v\in V} area(v)\cdot x_{v,i}&\leq Area_{max} &\forall v \in V\\
\sum_{v\in V} area(v)\cdot x_{v,i}&\geq Area_{min} &\forall v \in V\\
x_{e,i} &\leq x_{v,i}, &\forall e \in E, \forall v~\text{connected by}~e\\
\end{align}$
</ul>

In [31]:
# V - vertices; each vertex is one of the gates in the netlist; Each vertex is of type
# E - hyperedges; each edge/hyperedge is a tuple of vertices
# k - is the number of partitions
# (Amin, Amax) - minimum/maximum area for balance criterion
def partition(V, E, Amin, Amax, k = 2):
  import mip
  model = mip.Model(f"{k}-way partition")
  x = {u:[model.add_var(f"x_{u}_{i}", var_type = mip.BINARY) for i in range(k)] for u in V}
  x_e = {e:[model.add_var(f"x_{e}_{i}", var_type = mip.BINARY) for i in range(k)] for e in E}
  model.verbose = 0
  model.objective = mip.maximize(mip.xsum(x_e[e][i] for e in E for i in range(k)))

  for u in V:
    model += mip.xsum(x[u]) == 1

  for i in range(k):
    model += mip.xsum(V[u]._area*x[u][i] for u in V) >= Amin
    model += mip.xsum(V[u]._area*x[u][i] for u in V) <= Amax

  for e in E:
    for i in range(k):
      for v in E[e]:
        model += x_e[e][i] <= x[v._name][i]

  model.write(f"partition{k}.lp")
  model.optimize()
  sol = [list() for i in range(k)]

  if model.status == mip.OptimizationStatus.OPTIMAL:
    for u in V:
      for i in range(k):
        if round(x[u][i].x) == 1: sol[i].append(V[u])
    print(sol)
    return (sol, len(E)- model.objective.x)
  return None, None


In [36]:
from typing import DefaultDict
# bi-partition the input hypergraph using Fiduccia-Matheyses algorithm
# argument list is the same as the ILP version described above
# return value is a list of two lists and the number of cut hyperedges;
# each list is a partition comprising contained gates(vertices)
class Node:
  def __init__(self, name, area):
    self._name = name
    self._area = area
    self._ngbrs = []
    self._f = 0
    self._t = 0
    self._g = 0
    self._part = -1
  def __repr__(self):
    return self._name + f" ({round(self._area,2)})"

def updateNgbrs(V, E):
  import random
  V_temp = {V[v]._name: Node(V[v]._name, V[v]._area) for v in V}
  for v in V_temp:
    V_temp[v]._part = random.choice([0,1])
  for e in E:
    for node in E[e]:
      for nodeNxt in E[e]:
        if node != nodeNxt:
          V_temp[node._name]._ngbrs.append(V_temp[nodeNxt._name])
  return V_temp

def updateFTG(V_temp, E):
  for e in E:
    temp = []
    if len(E[e]) > 2:
      for node in E[e]:
        temp.append(node._part)
      partA = temp.count(0)
      partB = temp.count(1)
      if partA == 1:
        for node in E[e]:
          if node._part == 0:
            V_temp[E[e][0]._name]._f += 1
      elif partB == 1:
        for node in E[e]:
          if node._part == 1:
            V_temp[E[e][0]._name]._f += 1
      if partA == len(E[e]) or partB == len(E[e]):
        for node in E[e]:
          V_temp[E[e][0]._name]._t += 1
    else:
      if E[e][0]._part != E[e][1]._part:
        V_temp[E[e][0]._name]._f += 1
        V_temp[E[e][1]._name]._f += 1
      else:
        V_temp[E[e][0]._name]._t += 1
        V_temp[E[e][1]._name]._t += 1
  for v in V_temp:
    V_temp[v]._g = V_temp[v]._f - V_temp[v]._t
  return V_temp

def area(V_temp):
  area = 0
  for v in V_temp:
    if V_temp[v]._part == 0:
      area += V_temp[v]._area
  return area

def partitionFM(V, E, Amin, Amax):
  V_temp = updateNgbrs(V,E)
  E_temp = DefaultDict(list)
  for e in E:
    for v in E[e]:
      for n in V_temp:
        if v._name == V_temp[n]._name:
          E_temp[e].append(V_temp[n])
  V_temp = updateFTG(V_temp, E_temp)
  nodeList = []
  while len(nodeList) <= len(V):
    maxg = []
    for v in V_temp:
      maxg.append(V_temp[v]._g)

    maxg = list(set(maxg))
    maxg.sort(reverse=True)
    for m in maxg:
      booltoexit = False
      for v in V_temp:
        if V_temp[v]._g == m and (V_temp[v]._part == 1 and area(V_temp) + V_temp[v]._area <= Amax) and v not in nodeList:
          nodeList.append(V_temp[v])
          V_temp[v]._part = 0
          booltoexit = True
          break
        elif V_temp[v]._g == m and (V_temp[v]._part == 0 and area(V_temp) - V_temp[v]._area >= Amin):
          nodeList.append(V_temp[v])
          V_temp[v]._part = 1
          booltoexit = True
          break
      if booltoexit:
          break
    V_temp = updateFTG(V_temp, E_temp)

  A = []
  B = []
  for v in V_temp:
    if V_temp[v]._part == 0:
      A.append(V_temp[v])
    else:
      B.append(V_temp[v])
  cutList = []

  for e in E_temp:
    prevVpart = E_temp[e][0]._part
    for v in E_temp[e]:
      if v._part != prevVpart:
        cutList.append(e)
        break
  cutList = list(set(cutList))

  return ([A,B], len(cutList))

In [44]:
import time
for (lf,df) in [('sample.lef', 'sample.def'),('Nangate.lef', 'example.def')]: #
  V,E = loadNetlist(lf, df)
  print(E)
  Atotal = sum(V[u]._area for u in V)
  maxCellArea = max(V[u]._area for u in V)
  print("Total area :", Atotal, "delArea :", round(maxCellArea,2))
  for partfn in [partition, partitionFM]:
    t = time.time()
    sol, numcuts = partfn(V, E, Atotal/2 - maxCellArea, Atotal/2 + maxCellArea)
    print(f"{partfn.__name__} runtime : {time.time() - t}")
    if sol:
      print("number of cuts :", round(numcuts))
      for part in sol:
        print("area :", round(sum([x._area for x in part]),2), part)

{'net1237': [inst5638 (12.31), inst4678 (5.47)], 'net1240': [inst3502 (13.68), inst2015 (10.94)], 'net1233': [inst6050 (8.21), inst4189 (15.05)], 'net1236': [inst2908 (9.58), inst2591 (15.05)], 'net1234': [inst6458 (17.78), inst4597 (8.21)], 'net1232': [inst4382 (8.21), inst4062 (12.31)], 'net1231': [inst5821 (6.84), inst5275 (9.58)], 'net1239': [inst6286 (10.94), inst5333 (12.31)], 'net1235': [inst4183 (25.99), inst4132 (8.21)], 'net1238': [inst3444 (8.21), inst3428 (8.21)], 'net1230': [inst7234 (10.94), inst5195 (12.31)]}
Total area : 250.344 delArea : 25.99
[[inst2591 (15.05), inst2908 (9.58), inst3428 (8.21), inst3444 (8.21), inst4062 (12.31), inst4678 (5.47), inst4189 (15.05), inst4382 (8.21), inst5638 (12.31), inst5275 (9.58), inst5821 (6.84), inst6050 (8.21), inst5195 (12.31), inst7234 (10.94)], [inst2015 (10.94), inst3502 (13.68), inst4132 (8.21), inst4183 (25.99), inst4597 (8.21), inst5333 (12.31), inst6286 (10.94), inst6458 (17.78)]]
partition runtime : 0.01352071762084961
nu